# TITLE OF PROJECT

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time:
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:
* Video of 5-min Non-Technical Presentation:

## TABLE OF CONTENTS 

*Click to jump to matching Markdown Header.*<br><br>
 
- **[Introduction](#INTRODUCTION)<br>**
- **[OBTAIN](#OBTAIN)**<br>
- **[SCRUB](#SCRUB)**<br>
- **[EXPLORE](#EXPLORE)**<br>
- **[MODEL](#MODEL)**<br>
- **[iNTERPRET](#iNTERPRET)**<br>
- **[Conclusions/Recommendations](#CONCLUSIONS-&-RECOMMENDATIONS)<br>**
___

# INTRODUCTION

> Explain the point of your project and what question you are trying to answer with your modeling.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***



# OBTAIN

In [142]:
# import pandas as pd
# from sodapy import Socrata

# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# # client = Socrata("data.wa.gov", None)

# # Example authenticated client (needed for non-public datasets):
# client = Socrata('data.wa.gov',
#                  'C7QFByRWjTCvFfagF5Lt1QE50',
#                  username='427yub1zw87tmeol5eia062z',
#                  password='23yxf9ytpdtln3u8l4lxfv9u3y7gjidxh8kkw383xv7gbd4ogq')

# # First 2000 results, returned as JSON from API / converted to Python list of
# # dictionaries by sodapy.
# results = client.get("rpr4-cgyd", limit=2000)

# # Convert to pandas DataFrame
# results_df = pd.DataFrame.from_records(results)

## Obtaining data from state of Washington

### Accessing and Storing API Keys Locally

In [143]:
#option to run API calls
run_api_calls = False

In [144]:
#defining a function to open/load json files
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [145]:
#storing api keys in variables
keys = get_keys("/Users/berke/.secret/socrata_api_project_5.json")

api_key_socrata = keys['api_key']
app_token_socrata = keys['app_token']
api_key_secret_socrata = keys['api_key_secret']

### Sending requests and paginating

In [146]:
import requests
import time
import pandas as pd

if run_api_calls == True:
    #Defining/initializing API request headers and parameters
    headers = {'X-App-Token': app_token_socrata, 'username': api_key_socrata,
               'password': api_key_secret_socrata}
    params = {'$limit': '50000', '$offset': None}
    offset = list(range(0, 500000, 50000))

    #Requesting data from API and parsing results to a dictionary
    dfs = {}
    for number in offset:
        params['$offset'] = str(number)
        r = requests.get(f'https://data.wa.gov/resource/rpr4-cgyd.json?', 
                         headers=headers, params=params)
        dfs[f'df_{number}']=pd.DataFrame.from_records(r.json())
        time.sleep(1)

### Parsing results to a final compressed .csv file

In [147]:
if run_api_calls == True:
    #saving all pages as one csv file
    import os
    path = './data'
    output_file = os.path.join(path,'title_transactions.csv.gz')

    for key, df in dfs.items():
        if key == 'df_0':
            df_final = dfs['df_0']
        else:
            df_final = pd.concat([df_final, df], axis=0)

    df_final.to_csv(output_file, index=False, compression='gzip')    

### Importing data

In [148]:
#importing the data back into the notebook
df=pd.read_csv('data/title_transactions.csv.gz', compression='gzip', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433172 entries, 0 to 433171
Data columns (total 30 columns):
 #   Column                                                   Non-Null Count   Dtype  
---  ------                                                   --------------   -----  
 0   electric_vehicle_type                                    433172 non-null  object 
 1   vin_1_10                                                 433172 non-null  object 
 2   model_year                                               433172 non-null  int64  
 3   make                                                     433172 non-null  object 
 4   model                                                    433172 non-null  object 
 5   new_or_used_vehicle                                      433172 non-null  object 
 6   sale_price                                               433172 non-null  float64
 7   transaction_date                                         433172 non-null  object 
 8   transaction_ty

# SCRUB

## Changing type of 'transaction_date' to datetime

In [149]:
pd.set_option('max_columns', None)
df.head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_date,transaction_type,transaction_year,electric_vehicle_fee_paid,county,city,zip,electric_range,base_msrp,non_clean_alternative_fuel,date_of_vehicle_sale,vehicle_primary_use,state_of_residence,dol_vehicle_id,legislative_district,hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility,meets_2019_hb_2042_electric_range_requirement,meets_2019_hb_2042_sale_date_requirement,meets_2019_hb_2042_sale_price_value_requirement,odometer_reading,odometer_code,transportation_electrification_fee_paid,hybrid_vehicle_electrification_fee_paid
0,Battery Electric Vehicle (BEV),5YJXCBE2XG,2016,TESLA,Model X,New,102000.0,2017-01-31T00:00:00.000,Original Title,2017,Not Applicable,King,MEDINA,98039.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2016-12-27T00:00:00.000,Passenger,WA,348693390,48.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
1,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,0.0,2017-03-22T00:00:00.000,Original Registration,2017,No,King,SNOQUALMIE,98065.0,200,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,329748177,5.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
2,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,166250.0,2017-03-22T00:00:00.000,Original Title,2017,Not Applicable,King,SNOQUALMIE,98065.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-03T00:00:00.000,Passenger,WA,329748177,5.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
3,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,0.0,2017-03-24T00:00:00.000,Original Registration,2017,No,Island,OAK HARBOR,98277.0,53,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,242531271,10.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
4,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,32499.0,2017-03-24T00:00:00.000,Original Title,2017,Not Applicable,Island,OAK HARBOR,98277.0,53,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-11T00:00:00.000,Passenger,WA,242531271,10.0,TRANSACTION NOT ELIGIBLE: Sale before August 2019,True,False,True,99,Actual Mileage,Not Applicable,Not Applicable


In [150]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df.head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_date,transaction_type,transaction_year,electric_vehicle_fee_paid,county,city,zip,electric_range,base_msrp,non_clean_alternative_fuel,date_of_vehicle_sale,vehicle_primary_use,state_of_residence,dol_vehicle_id,legislative_district,hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility,meets_2019_hb_2042_electric_range_requirement,meets_2019_hb_2042_sale_date_requirement,meets_2019_hb_2042_sale_price_value_requirement,odometer_reading,odometer_code,transportation_electrification_fee_paid,hybrid_vehicle_electrification_fee_paid
0,Battery Electric Vehicle (BEV),5YJXCBE2XG,2016,TESLA,Model X,New,102000.0,2017-01-31,Original Title,2017,Not Applicable,King,MEDINA,98039.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2016-12-27T00:00:00.000,Passenger,WA,348693390,48.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
1,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,0.0,2017-03-22,Original Registration,2017,No,King,SNOQUALMIE,98065.0,200,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,329748177,5.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
2,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,166250.0,2017-03-22,Original Title,2017,Not Applicable,King,SNOQUALMIE,98065.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-03T00:00:00.000,Passenger,WA,329748177,5.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable
3,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,0.0,2017-03-24,Original Registration,2017,No,Island,OAK HARBOR,98277.0,53,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,242531271,10.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No
4,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,32499.0,2017-03-24,Original Title,2017,Not Applicable,Island,OAK HARBOR,98277.0,53,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-11T00:00:00.000,Passenger,WA,242531271,10.0,TRANSACTION NOT ELIGIBLE: Sale before August 2019,True,False,True,99,Actual Mileage,Not Applicable,Not Applicable


## Feature Engineering "m/y" column

In [151]:
df['m/y'] = df['transaction_date'].dt.strftime("%m-%Y")

## Adjusting the index of df

In [152]:
df.set_index('transaction_date', inplace=True)
df.head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,transaction_year,electric_vehicle_fee_paid,county,city,zip,electric_range,base_msrp,non_clean_alternative_fuel,date_of_vehicle_sale,vehicle_primary_use,state_of_residence,dol_vehicle_id,legislative_district,hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility,meets_2019_hb_2042_electric_range_requirement,meets_2019_hb_2042_sale_date_requirement,meets_2019_hb_2042_sale_price_value_requirement,odometer_reading,odometer_code,transportation_electrification_fee_paid,hybrid_vehicle_electrification_fee_paid,m/y
transaction_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-31,Battery Electric Vehicle (BEV),5YJXCBE2XG,2016,TESLA,Model X,New,102000.0,Original Title,2017,Not Applicable,King,MEDINA,98039.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2016-12-27T00:00:00.000,Passenger,WA,348693390,48.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable,01-2017
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,0.0,Original Registration,2017,No,King,SNOQUALMIE,98065.0,200,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,329748177,5.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No,03-2017
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,166250.0,Original Title,2017,Not Applicable,King,SNOQUALMIE,98065.0,200,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-03T00:00:00.000,Passenger,WA,329748177,5.0,TRANSACTION NOT ELIGIBLE: Sale before August 2...,True,False,False,50,Actual Mileage,Not Applicable,Not Applicable,03-2017
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,0.0,Original Registration,2017,No,Island,OAK HARBOR,98277.0,53,0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",NaN,Passenger,WA,242531271,10.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No,03-2017
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,32499.0,Original Title,2017,Not Applicable,Island,OAK HARBOR,98277.0,53,0,CANNOT DETERMINE ELIGIBILITY: Base MSRP has no...,2017-03-11T00:00:00.000,Passenger,WA,242531271,10.0,TRANSACTION NOT ELIGIBLE: Sale before August 2019,True,False,True,99,Actual Mileage,Not Applicable,Not Applicable,03-2017


## Column Exploration

There is great information in this dataframe but there seems to also be irrelevant information for our purposes. In order to clean up the dataframe we need to know which columns we should keep and which ones we should drop. 

### Dropping columns that are for Governmental Use

To start off, there are several columns that seem to be of use for the government or the vehicle's owner but is irrelevant for us in answering the business question at hand. Some examples include the following along with the description provided in the API:

**'meets_2019_hb_2042_electric_range_requirement':** True = The vehicle model can travel 30 miles or more solely on electricity. Otherwise, False.

**'meets_2019_hb_2042_sale_date_requirement':** True = The vehicle was sold on or after August 1, 2019. Otherwise, False.

**'meets_2019_hb_2042_sale_price_value_requirement':** True = The vehicle sale price/value was 45,000 or less for new vehicles, or 30,000 or less for used vehicles. Otherwise, False.

**'hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility':** Shows if vehicle title transactions were eligible for the sales tax exemption authorized by House Bill 2042 during the 2019 Legislative Session. If not eligible, reasons are provided.

**'non_clean_alternative_fuel':** Shows if new vehicle title transactions were eligible for the sales tax exemption authorized by House Bill 2778 during the 2015 Legislative Session. If not eligible, reasons are provided.

**'transportation_electrification_fee_paid':** The Transportation Electrification Fee is charged to some electric vehicles when they renew their registration. This indicates if it was collected during the transaction

**'hybrid_vehicle_electrification_fee_paid':** The Hybrid Vehicle Electrification Fee is charged to some electric vehicles when they renew their registration. This indicates if it was collected during the transaction.

**'legislative_district':** The specific section of Washington State that the vehicle's owner resides in, as represented in the state legislature.

**'electric_vehicle_fee_paid':** The Electric Vehicle Fee is charged to some electric vehicles when they renew their registration. This indicates if it was collected during the transaction.

We can go ahead and drop these columns since these are mostly about whether a fee was paid or if the vehicle met certain governmental requirements for tax credits or otherwise.

In [153]:
drop_cols = ['electric_vehicle_fee_paid',
             'hb_2042_clean_alternative_fuel_vehicle_cafv_eligibility',
             'meets_2019_hb_2042_electric_range_requirement', 
             'meets_2019_hb_2042_sale_date_requirement', 
             'meets_2019_hb_2042_sale_price_value_requirement', 
             'transportation_electrification_fee_paid', 
             'hybrid_vehicle_electrification_fee_paid', 'legislative_district', 
             'non_clean_alternative_fuel']
df.drop(drop_cols, axis=1, inplace=True)
df.head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,transaction_year,county,city,zip,electric_range,base_msrp,date_of_vehicle_sale,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,,,,
2017-01-31,Battery Electric Vehicle (BEV),5YJXCBE2XG,2016,TESLA,Model X,New,102000.0,Original Title,2017,King,MEDINA,98039.0,200,0,2016-12-27T00:00:00.000,Passenger,WA,348693390,50,Actual Mileage,01-2017
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,0.0,Original Registration,2017,King,SNOQUALMIE,98065.0,200,0,NaN,Passenger,WA,329748177,0,Odometer reading is not collected at time of r...,03-2017
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,166250.0,Original Title,2017,King,SNOQUALMIE,98065.0,200,0,2017-03-03T00:00:00.000,Passenger,WA,329748177,50,Actual Mileage,03-2017
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,0.0,Original Registration,2017,Island,OAK HARBOR,98277.0,53,0,NaN,Passenger,WA,242531271,0,Odometer reading is not collected at time of r...,03-2017
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,32499.0,Original Title,2017,Island,OAK HARBOR,98277.0,53,0,2017-03-11T00:00:00.000,Passenger,WA,242531271,99,Actual Mileage,03-2017


### Dropping columns with redundant information

In [154]:
df['date_of_vehicle_sale'].isna().sum()

289770

Since we have the full transaction date in our index the 'transaction_year' column is redundant. Additionally, the date_of_vehicle_sale column seems to be missing quite a bit of information. Where there is data, the difference between the transaction date and the actual vehicle sale seems to be around a couple weeks so the difference is not meaningful for our purposes.

Also, we already have the sale price of the vehicle at the time of the transaction so the base MSRP column is irrelevant. So we can go ahead and drop these columns from our dataframe as well. We will be keeping the rest of the columns for the time being for EDA purposes.

In [155]:
drop_cols = ['transaction_year', 'base_msrp', 'date_of_vehicle_sale']
df.drop(drop_cols, axis=1, inplace=True)
df.head()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,
2017-01-31,Battery Electric Vehicle (BEV),5YJXCBE2XG,2016,TESLA,Model X,New,102000.0,Original Title,King,MEDINA,98039.0,200,Passenger,WA,348693390,50,Actual Mileage,01-2017
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,0.0,Original Registration,King,SNOQUALMIE,98065.0,200,Passenger,WA,329748177,0,Odometer reading is not collected at time of r...,03-2017
2017-03-22,Battery Electric Vehicle (BEV),5YJXCBE49H,2017,TESLA,Model X,New,166250.0,Original Title,King,SNOQUALMIE,98065.0,200,Passenger,WA,329748177,50,Actual Mileage,03-2017
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,0.0,Original Registration,Island,OAK HARBOR,98277.0,53,Passenger,WA,242531271,0,Odometer reading is not collected at time of r...,03-2017
2017-03-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S59H,2017,CHEVROLET,Volt,New,32499.0,Original Title,Island,OAK HARBOR,98277.0,53,Passenger,WA,242531271,99,Actual Mileage,03-2017


In [156]:
df=df[(df['transaction_type']=='Original Title') | 
      (df['transaction_type']=='Transfer Title')]
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 121542 entries, 2017-01-31 to 2017-05-01
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   electric_vehicle_type  121542 non-null  object 
 1   vin_1_10               121542 non-null  object 
 2   model_year             121542 non-null  int64  
 3   make                   121542 non-null  object 
 4   model                  121542 non-null  object 
 5   new_or_used_vehicle    121542 non-null  object 
 6   sale_price             121542 non-null  float64
 7   transaction_type       121542 non-null  object 
 8   county                 121534 non-null  object 
 9   city                   121514 non-null  object 
 10  zip                    121537 non-null  float64
 11  electric_range         121542 non-null  int64  
 12  vehicle_primary_use    121542 non-null  object 
 13  state_of_residence     121540 non-null  object 
 14  dol_vehicle_id      

## Addressing Duplicates

Now that we have a dataframe that shows only the purchases of new and used cars and none of the registration information, we can go ahead and address duplicates. One note here is that we are not going to be deleting duplicates based on a car's ID number since we would like to keep track of each car as they are getting sold to people in different counties.

In [157]:
df[df.duplicated()]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,
2018-04-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RB6S51J,2018,CHEVROLET,Volt,New,33108.0,Original Title,Snohomish,EDMONDS,98020.0,53,Passenger,WA,165661520,5,Actual Mileage,04-2018
2017-03-15,Plug-in Hybrid Electric Vehicle (PHEV),1FADP5CU6E,2014,FORD,C-max,Used,15000.0,Original Title,Grant,EPHRATA,98823.0,19,Passenger,WA,154225991,18388,Actual Mileage,03-2017
2019-01-02,Battery Electric Vehicle (BEV),5YJXCBE27J,2018,TESLA,Model X,New,114700.0,Original Title,King,SEATTLE,98102.0,238,Passenger,WA,116650978,50,Actual Mileage,01-2019
2019-04-05,Battery Electric Vehicle (BEV),5YJ3E1EA9K,2019,TESLA,Model 3,New,46200.0,Original Title,King,SEATTLE,98102.0,220,Passenger,WA,209719674,50,Actual Mileage,04-2019
2018-07-13,Battery Electric Vehicle (BEV),5YJ3E1EA5J,2018,TESLA,Model 3,New,57500.0,Original Title,Clark,CAMAS,98607.0,215,Passenger,WA,338187300,50,Actual Mileage,07-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-17,Battery Electric Vehicle (BEV),1N4AZ0CP5F,2015,NISSAN,Leaf,Used,9950.0,Original Title,King,KENMORE,98028.0,84,Passenger,WA,117975602,30068,Actual Mileage,01-2017
2017-01-13,Plug-in Hybrid Electric Vehicle (PHEV),1G1RB6E49D,2013,CHEVROLET,Volt,Used,0.0,Original Title,Kittitas,ELLENSBURG,98926.0,38,Passenger,WA,349914309,30899,Actual Mileage,01-2017
2017-03-08,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6E42D,2013,CHEVROLET,Volt,Used,13992.0,Transfer Title,Pierce,GRAHAM,98338.0,38,Passenger,WA,136639037,25903,Actual Mileage,03-2017


We have 1287 rows of data that is duplicated. In order to make sure that these are indeed duplicates with no differences we can check a few of the vehicles.

In [158]:
df[df['dol_vehicle_id']==165661520]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,
2018-04-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RB6S51J,2018,CHEVROLET,Volt,New,33108.0,Original Title,Snohomish,EDMONDS,98020.0,53,Passenger,WA,165661520,5,Actual Mileage,04-2018
2018-04-24,Plug-in Hybrid Electric Vehicle (PHEV),1G1RB6S51J,2018,CHEVROLET,Volt,New,33108.0,Original Title,Snohomish,EDMONDS,98020.0,53,Passenger,WA,165661520,5,Actual Mileage,04-2018


In [159]:
df[df['dol_vehicle_id']==154225991]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,
2017-03-15,Plug-in Hybrid Electric Vehicle (PHEV),1FADP5CU6E,2014,FORD,C-max,Used,15000.0,Original Title,Grant,EPHRATA,98823.0,19,Passenger,WA,154225991,18388,Actual Mileage,03-2017
2017-03-15,Plug-in Hybrid Electric Vehicle (PHEV),1FADP5CU6E,2014,FORD,C-max,Used,15000.0,Original Title,Grant,EPHRATA,98823.0,19,Passenger,WA,154225991,18388,Actual Mileage,03-2017


Data in each column for these transactions seem to be exactly the same so we can go ahead and drop these rows.

In [160]:
df.drop_duplicates(inplace=True)
#verifying that duplicates have been removed
len(df[df.duplicated()])

0

In [161]:
df[df['dol_vehicle_id']==154225991]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,
2017-03-15,Plug-in Hybrid Electric Vehicle (PHEV),1FADP5CU6E,2014,FORD,C-max,Used,15000.0,Original Title,Grant,EPHRATA,98823.0,19,Passenger,WA,154225991,18388,Actual Mileage,03-2017


In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 120255 entries, 2017-01-31 to 2017-05-01
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   electric_vehicle_type  120255 non-null  object 
 1   vin_1_10               120255 non-null  object 
 2   model_year             120255 non-null  int64  
 3   make                   120255 non-null  object 
 4   model                  120255 non-null  object 
 5   new_or_used_vehicle    120255 non-null  object 
 6   sale_price             120255 non-null  float64
 7   transaction_type       120255 non-null  object 
 8   county                 120247 non-null  object 
 9   city                   120227 non-null  object 
 10  zip                    120250 non-null  float64
 11  electric_range         120255 non-null  int64  
 12  vehicle_primary_use    120255 non-null  object 
 13  state_of_residence     120253 non-null  object 
 14  dol_vehicle_id      

## Checking for and Addressing Null Values

Next, we will be checking for and addressing null values in the dataset.

In [164]:
df.isna().sum()

electric_vehicle_type     0
vin_1_10                  0
model_year                0
make                      0
model                     0
new_or_used_vehicle       0
sale_price                0
transaction_type          0
county                    8
city                     28
zip                       5
electric_range            0
vehicle_primary_use       0
state_of_residence        2
dol_vehicle_id            0
odometer_reading          0
odometer_code             0
m/y                       0
dtype: int64

Here, we see that 4 columns: 'county', 'city', 'zip' and 'state_of_residence' all have null values. We can check the display the null values in each column to see if we can fill them in.

In [170]:
df[df['county'].isna()]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,
2019-04-16,Battery Electric Vehicle (BEV),5YJXCDE46J,2018,TESLA,Model X,New,105000.0,Original Title,NaN,ALISO VIEJO,92656.0,238,Passenger,CA,229979662,1195,Actual Mileage,04-2019
2019-12-26,Plug-in Hybrid Electric Vehicle (PHEV),KMHC65LD3K,2019,HYUNDAI,Ioniq,New,24684.0,Original Title,NaN,MC GUIRE AFB,8641.0,29,Passenger,NJ,8189709,9,Actual Mileage,12-2019
2021-03-19,Plug-in Hybrid Electric Vehicle (PHEV),3FA6P0PUXG,2016,FORD,Fusion,Used,0.0,Original Title,NaN,FPO,96620.0,19,Passenger,CA,148972407,52000,Actual Mileage,03-2021
2018-04-13,Plug-in Hybrid Electric Vehicle (PHEV),3FA6P0PU8J,2018,FORD,Fusion,New,36943.0,Original Title,NaN,TULSA,74110.0,21,Passenger,OK,350141788,5,Actual Mileage,04-2018
2014-09-23,Battery Electric Vehicle (BEV),JN1AZ0CP1B,2011,NISSAN,Leaf,Used,0.0,Transfer Title,NaN,NaN,NaN,73,Passenger,NaN,133789183,46005,Actual Mileage,09-2014
2017-12-18,Battery Electric Vehicle (BEV),5YJSA1S14E,2014,TESLA,MODEL S,Used,0.0,Transfer Title,NaN,WESTMONT,60559.0,208,Passenger,IL,145245145,39620,Actual Mileage,12-2017
2017-10-04,Battery Electric Vehicle (BEV),5YJXCBE23H,2017,TESLA,Model X,New,115750.0,Original Title,NaN,LOS ALTOS,94022.0,200,Passenger,CA,348925216,50,Actual Mileage,10-2017
2018-02-28,Plug-in Hybrid Electric Vehicle (PHEV),5UXKT0C38H,2017,BMW,X5,Used,55000.0,Transfer Title,NaN,NaN,NaN,14,Passenger,NaN,111058636,6919,Actual Mileage,02-2018


The county information is missing on vehicles with owners residing in different states. Our analysis is focused on the Washington State and without in-state address information these data points are not going to be useful so we can go ahead and drop these from our dataframe.

In [176]:
#dropping null values
df.dropna(subset=['county'], inplace=True)

In [178]:
#verifying that the null values have been dropped
df[df['county'].isna()]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,


As discussed above, the data analysis is focused on Washington State, hence there is no reason to keep vehicles with owners residing outside of WA since we don't have their in-state locations. Additionally, this will remove all null values from the state_of_residence column.

In [179]:
df=df[df['state_of_residence']=='WA']

In [180]:
df[df['city'].isna()]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,
2017-06-13,Battery Electric Vehicle (BEV),5YJXCBE40H,2017,TESLA,Model X,New,161500.00,Original Title,Chelan,NaN,98847.0,200,Passenger,WA,105297153,50,Actual Mileage,06-2017
2013-09-10,Battery Electric Vehicle (BEV),1N4AZ0CPXD,2013,NISSAN,Leaf,New,0.00,Original Title,Whatcom,NaN,98266.0,75,Passenger,WA,162570523,16,Actual Mileage,09-2013
2017-02-21,Battery Electric Vehicle (BEV),1N4BZ0CP4G,2016,NISSAN,Leaf,New,30200.00,Original Title,San Juan,NaN,98280.0,84,Passenger,WA,119455920,100,Actual Mileage,02-2017
2017-04-05,Battery Electric Vehicle (BEV),5YJXCAE23H,2017,TESLA,Model X,New,122000.00,Original Title,San Juan,NaN,98243.0,200,Passenger,WA,266866526,50,Actual Mileage,04-2017
2019-12-16,Battery Electric Vehicle (BEV),1N4BZ1CP0K,2019,NISSAN,Leaf,New,41800.00,Original Title,San Juan,NaN,98280.0,150,Passenger,WA,3653366,210,Actual Mileage,12-2019
2017-07-12,Battery Electric Vehicle (BEV),JN1AZ0CPXC,2012,NISSAN,Leaf,Used,7000.00,Transfer Title,San Juan,NaN,98243.0,73,Passenger,WA,208715517,35077,Actual Mileage,07-2017
2020-09-22,Battery Electric Vehicle (BEV),1G1FY6S09L,2020,CHEVROLET,Bolt EV,New,30204.24,Original Title,Skagit,NaN,98221.0,259,Exempt (State/County/Local/Tribal),WA,124578051,4,Actual Mileage,09-2020
2021-03-31,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z4C55E,2014,BMW,i3,Used,12300.00,Original Title,San Juan,NaN,98243.0,72,Passenger,WA,148276376,58562,Actual Mileage,03-2021
2017-12-04,Battery Electric Vehicle (BEV),1N4BZ0CP7H,2017,NISSAN,Leaf,New,38351.00,Original Title,San Juan,NaN,98280.0,107,Passenger,WA,208825039,28,Actual Mileage,12-2017


Since we are mainly concerned about the vehicles on a county level the null values in the city column are not concerning; however, we can change the null values with 'Unknown' as a placeholder.

In [182]:
#filling null values
df['city'].fillna('Unknown', inplace=True)
#verifying that null values have been replaced
df[df['city'].isna()]

C:\Users\berke\anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,


In [184]:
df[df['zip'].isna()]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
transaction_date,,,,,,,,,,,,,,,,,,
2019-05-17,Battery Electric Vehicle (BEV),KL8CL6S03G,2016,CHEVROLET,Spark,Used,9991.0,Original Title,Whitman,PALOUSE,NaN,82,Passenger,WA,236532600,10424,Actual Mileage,05-2019
2020-06-09,Battery Electric Vehicle (BEV),5YJ3E1EC9L,2020,TESLA,Model 3,New,59290.0,Original Title,Benton,RICHLAND,NaN,308,Passenger,WA,109116017,25,Actual Mileage,06-2020
2020-02-12,Plug-in Hybrid Electric Vehicle (PHEV),KNDCC3LC8K,2019,KIA,NIRO PLUG-IN HYBRID,New,30695.0,Original Title,Lewis,ONALASKA,NaN,26,Passenger,WA,102086750,41,Actual Mileage,02-2020


Once again, since we have the county information for these vehicles, we can replace the null values in the zip columns with 'Unknown' once again.

In [185]:
df['zip'].fillna('Unknown', inplace=True)

C:\Users\berke\anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [186]:
#verifying that no null values are remaining
df.isna().sum()

electric_vehicle_type    0
vin_1_10                 0
model_year               0
make                     0
model                    0
new_or_used_vehicle      0
sale_price               0
transaction_type         0
county                   0
city                     0
zip                      0
electric_range           0
vehicle_primary_use      0
state_of_residence       0
dol_vehicle_id           0
odometer_reading         0
odometer_code            0
m/y                      0
dtype: int64

In [187]:
df['county'].value_counts()

King            65437
Snohomish       12975
Pierce           9424
Clark            6368
Thurston         4292
Kitsap           4242
Whatcom          3073
Spokane          2693
Island           1316
Benton           1316
Skagit           1278
Clallam           741
Jefferson         715
San Juan          694
Cowlitz           562
Mason             562
Chelan            557
Yakima            530
Lewis             458
Grays Harbor      419
Franklin          354
Grant             286
Kittitas          282
Walla Walla       265
Douglas           209
Whitman           163
Klickitat         158
Pacific           142
Stevens           129
Okanogan          112
Skamania          103
Asotin             50
Adams              33
Pend Oreille       29
Wahkiakum          28
Lincoln            27
Ferry              16
Columbia           13
Garfield            2
Name: county, dtype: int64

## GROUPBY - IN PROGRESS

In [126]:
df_grouped = df.groupby(by=['m/y', 'dol_vehicle_id']).last()

In [121]:
df[df['dol_vehicle_id']==477791625]

,transaction_date,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code,m/y
53171,2019-09-13,Battery Electric Vehicle (BEV),5YJ3E1EA2K,2019,TESLA,Model 3,New,41690.0,Original Title,Clark,VANCOUVER,98683.0,220,Passenger,WA,477791625,50,Actual Mileage,09-2019
63258,2020-02-25,Battery Electric Vehicle (BEV),5YJ3E1EA2K,2019,TESLA,Model 3,Used,43987.0,Transfer Title,Snohomish,BOTHELL,98021.0,220,Passenger,WA,477791625,4803,Actual Mileage,02-2020
111792,2020-12-31,Battery Electric Vehicle (BEV),5YJ3E1EA2K,2019,TESLA,Model 3,Used,30000.0,Transfer Title,Skagit,BURLINGTON,98233.0,220,Passenger,WA,477791625,10715,Actual Mileage,12-2020


In [72]:
df_new_cars = df[(df['transaction_type']=='Original Title')]

In [73]:
df_new_cars.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 96961 entries, 2017-01-31 to 2017-05-01
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   electric_vehicle_type  96961 non-null  object 
 1   vin_1_10               96961 non-null  object 
 2   model_year             96961 non-null  int64  
 3   make                   96961 non-null  object 
 4   model                  96961 non-null  object 
 5   new_or_used_vehicle    96961 non-null  object 
 6   sale_price             96961 non-null  float64
 7   transaction_type       96961 non-null  object 
 8   county                 96956 non-null  object 
 9   city                   96939 non-null  object 
 10  zip                    96958 non-null  float64
 11  electric_range         96961 non-null  int64  
 12  vehicle_primary_use    96961 non-null  object 
 13  state_of_residence     96961 non-null  object 
 14  dol_vehicle_id         96961 non-null

In [25]:
len(df_new_cars['dol_vehicle_id'].unique())

91296

In [26]:
df_new_cars['transaction_type'].value_counts()

Original Title    96961
Name: transaction_type, dtype: int64

In [27]:
df_new_cars[df_new_cars['dol_vehicle_id'].duplicated()]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code
transaction_date,,,,,,,,,,,,,,,,,
2018-06-21,Plug-in Hybrid Electric Vehicle (PHEV),2C4RC1N70J,2018,CHRYSLER,Pacifica,New,46100.0,Original Title,Kitsap,BREMERTON,98311.0,33,Passenger,WA,185017913,21,Actual Mileage
2017-07-12,Plug-in Hybrid Electric Vehicle (PHEV),1G1RE6E48E,2014,CHEVROLET,Volt,Used,14900.0,Original Title,Thurston,OLYMPIA,98513.0,38,Passenger,WA,332189693,45700,Actual Mileage
2017-06-30,Plug-in Hybrid Electric Vehicle (PHEV),1G1RA6S54H,2017,CHEVROLET,Volt,New,34095.0,Original Title,Pierce,ROY,98580.0,53,Passenger,WA,300952816,389,Actual Mileage
2017-12-01,Plug-in Hybrid Electric Vehicle (PHEV),1G1RB6S5XH,2017,CHEVROLET,Volt,New,37570.0,Original Title,Snohomish,LYNNWOOD,98037.0,53,Passenger,WA,118520461,99,Actual Mileage
2017-09-22,Plug-in Hybrid Electric Vehicle (PHEV),JTDKN3DP1F,2015,TOYOTA,Prius Plug-in,Used,17777.0,Original Title,Whitman,PULLMAN,99163.0,6,Passenger,WA,313766127,33451,Actual Mileage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-25,Battery Electric Vehicle (BEV),5YJSA1E21G,2016,TESLA,Model S,New,91700.0,Original Title,Snohomish,LYNNWOOD,98036.0,210,Passenger,WA,348359952,50,Actual Mileage
2017-02-02,Battery Electric Vehicle (BEV),5YJSA1E26G,2016,TESLA,Model S,New,76700.0,Original Title,King,REDMOND,98053.0,210,Passenger,WA,349789597,50,Actual Mileage
2017-03-16,Battery Electric Vehicle (BEV),5YJSA1E25H,2017,TESLA,Model S,New,106950.0,Original Title,Whatcom,CUSTER,98240.0,210,Passenger,WA,113869640,50,Actual Mileage


In [28]:
df_new_cars[df_new_cars['dol_vehicle_id']==185017913]

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code
transaction_date,,,,,,,,,,,,,,,,,
2018-06-21,Plug-in Hybrid Electric Vehicle (PHEV),2C4RC1N70J,2018,CHRYSLER,Pacifica,New,46100.0,Original Title,Kitsap,SILVERDALE,98315.0,33,Passenger,WA,185017913,21,Actual Mileage
2018-06-21,Plug-in Hybrid Electric Vehicle (PHEV),2C4RC1N70J,2018,CHRYSLER,Pacifica,New,46100.0,Original Title,Kitsap,BREMERTON,98311.0,33,Passenger,WA,185017913,21,Actual Mileage


In [82]:
df[df['dol_vehicle_id']==127890535].sort_index()

,electric_vehicle_type,vin_1_10,model_year,make,model,new_or_used_vehicle,sale_price,transaction_type,county,city,zip,electric_range,vehicle_primary_use,state_of_residence,dol_vehicle_id,odometer_reading,odometer_code
transaction_date,,,,,,,,,,,,,,,,,
2017-04-13,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z8C3XH,2017,BMW,i3,New,0.0,Original Registration,King,ISSAQUAH,98029.0,97,Passenger,WA,127890535,0,Odometer reading is not collected at time of r...
2017-04-13,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z8C3XH,2017,BMW,i3,New,46350.0,Original Title,King,ISSAQUAH,98029.0,97,Passenger,WA,127890535,18,Actual Mileage
2018-02-27,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z8C3XH,2017,BMW,i3,Used,0.0,Registration Renewal,King,ISSAQUAH,98029.0,97,Passenger,WA,127890535,0,Odometer reading is not collected at time of r...
2019-03-15,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z8C3XH,2017,BMW,i3,Used,0.0,Registration Renewal,King,ISSAQUAH,98029.0,97,Passenger,WA,127890535,0,Odometer reading is not collected at time of r...
2020-06-01,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z8C3XH,2017,BMW,i3,Used,0.0,Registration at time of Transfer,King,SEATTLE,98133.0,97,Passenger,WA,127890535,0,Odometer reading is not collected at time of r...
2020-06-01,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z8C3XH,2017,BMW,i3,Used,20495.0,Transfer Title,King,SEATTLE,98133.0,97,Passenger,WA,127890535,29852,Actual Mileage
2020-06-01,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z8C3XH,2017,BMW,i3,Used,0.0,Registration at time of Transfer,King,SHORELINE,98155.0,97,Passenger,WA,127890535,0,Odometer reading is not collected at time of r...
2020-06-01,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z8C3XH,2017,BMW,i3,Used,20495.0,Transfer Title,King,SHORELINE,98155.0,97,Passenger,WA,127890535,29852,Actual Mileage
2021-05-17,Plug-in Hybrid Electric Vehicle (PHEV),WBY1Z8C3XH,2017,BMW,i3,Used,0.0,Registration Renewal,King,SHORELINE,98155.0,97,Passenger,WA,127890535,0,Odometer reading is not collected at time of r...


## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

# EXPLORE

In [66]:
df['electric_vehicle_type'].value_counts()

Battery Electric Vehicle (BEV)            302756
Plug-in Hybrid Electric Vehicle (PHEV)    130416
Name: electric_vehicle_type, dtype: int64

In [67]:
df['new_or_used_vehicle'].value_counts()

Used    275022
New     158150
Name: new_or_used_vehicle, dtype: int64

In [68]:
df['vehicle_primary_use'].value_counts()

Passenger                             429023
Exempt (State/County/Local/Tribal)      2566
Commercial                               755
Truck                                    615
For Hire                                 134
Medium-Speed Electric Vehicle             47
Neighborhood Electric Vehicle             13
Cab                                        5
Medium-Speed Electric Truck                3
Taxi Cab                                   3
Private School                             3
Farm Use                                   3
Farm Exempt                                2
Name: vehicle_primary_use, dtype: int64

In [69]:
df['state_of_residence'].value_counts()

WA    432652
CA        86
VA        68
TX        53
OR        41
MD        28
GA        25
MI        18
NV        15
NJ        14
IL        14
NY        14
CO        13
AK        12
FL        12
NC        10
CT        10
OK         8
AL         8
KS         7
AZ         6
MA         6
UT         5
DC         5
ID         5
HI         4
MT         4
OH         3
TN         3
LA         3
SC         3
MN         2
DE         2
AR         2
KY         2
IN         1
PA         1
MO         1
WY         1
WI         1
Name: state_of_residence, dtype: int64

In [70]:
df['make'].value_counts()

TESLA                   133069
NISSAN                  109494
CHEVROLET                57133
FORD                     31540
BMW                      20725
KIA                      19929
TOYOTA                   18223
HYUNDAI                   6384
AUDI                      5858
CHRYSLER                  4786
FIAT                      4282
VOLKSWAGEN                4222
VOLVO                     3375
HONDA                     3133
MITSUBISHI                2408
PORSCHE                   1976
SMART                     1968
MERCEDES-BENZ             1358
MINI                       987
CADILLAC                   779
JAGUAR                     630
FISKER                     252
SUBARU                     152
JEEP                       118
LAND ROVER                 115
AZURE DYNAMICS              71
WHEEGO ELECTRIC CARS        61
LINCOLN                     60
TH!NK                       46
POLESTAR                    20
DODGE                        7
BUBBLE BUDDY                 6
PLYMOUTH

In [71]:
df['transaction_type'].value_counts()

Registration Renewal                191199
Original Registration                97449
Original Title                       96961
Transfer Title                       24581
Registration at time of Transfer     22982
Name: transaction_type, dtype: int64

# MODEL

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

# iNTERPRET

Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

# CONCLUSIONS & RECOMMENDATIONS

Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***